In [ ]:
!pip install -q pdfplumber
!pip install -q  langchain-text-splitters

from google.colab import files
import pdfplumber
import sys # Import sys for SystemExit
from langchain_text_splitters import RecursiveCharacterTextSplitter

print("Upload your PDF file")
uploaded = files.upload()

if not uploaded:
    print("No file was uploaded. Please upload a PDF file to proceed.", file=sys.stderr)
    raise SystemExit("No file uploaded.")

pdf_file = list(uploaded.keys())[0]

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        return text
    except pdfplumber.pdf.PdfminerException as e:
        print(f"Error: The file '{pdf_path}' could not be processed as a PDF. It might be corrupted or not a PDF file. Original error: {e}", file=sys.stderr)
        return None

pdf_text = extract_text_from_pdf(pdf_file)

if pdf_text is not None:
    print("\nPDF Text Extracted Successfully\n")
    print(pdf_text[:2000])

    # Define chunking parameters
    chunk_size = 1000 # Number of characters in each chunk
    chunk_overlap = 200 # Number of characters to overlap between chunks

    # Initialize the text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        add_start_index=True,
    )

    # Split the text into chunks
    chunks = text_splitter.create_documents([pdf_text])

    print(f"\nCreated {len(chunks)} chunks.")
    print("\n--- First 3 Chunks ---\n")
    for i, chunk in enumerate(chunks[:3]):
        print(f"Chunk {i+1} (Length: {len(chunk.page_content)}):\n{chunk.page_content[:500]}...\n")

else:
    print("\nFailed to extract text. Please ensure you uploaded a valid PDF file.")


Upload your PDF file


Saving my_histroy.txt.pdf to my_histroy.txt (1).pdf

PDF Text Extracted Successfully

>>> I want you to act as an Unbiased Behavioral Analyst and Mirror. I will tell
you about my past, my feelings, and my a... ctions. Do not be 'nice' for the
sake of it. I want you to judge my character based on my patterns. Tell me who I
am... from an outside perspective, how others likely perceive me, and where I
am lying to myself. Be brutally honest, obj... ective, and analytical. Your goal
is to help me see my true self so I can decide how to change.
Alright, Dolphin here, ready to provide you with unbiased analysis. Please share
your past experiences, feelings,
and actions, and let's start building a picture of who you are.
>>> so before we start , let me make few things clear , after i provide all the
data , i want you to recall everything ,... consider them and i may ask you to
list them as well and finally judge me psyschologically and give a proper
unbais... ed opinion and judgement out of th

In [ ]:
!pip install -q openai-whisper
import whisper
from google.colab import files
import textwrap # Import textwrap for formatting

print("Upload audio file (wav/mp3/mp4/m4a)")
uploaded = files.upload()
audio_file = list(uploaded.keys())[0]

print(f"Transcribing {audio_file}...Please wait....")
model = whisper.load_model("medium")
result = model.transcribe(audio_file)

print("\n---Transcription---\n")
# Wrap the text to a specific width for better readability
wrapped_text = textwrap.fill(result["text"], width=80)
print(wrapped_text)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 14.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Upload audio file (wav/mp3/mp4/m4a)


Saving Glass Animals - Heat Waves.mp3 to Glass Animals - Heat Waves.mp3
Transcribing Glass Animals - Heat Waves.mp3...Please wait....


100%|█████████████████████████████████████| 1.42G/1.42G [00:19<00:00, 78.8MiB/s]



---Transcription---

 Won't you roll with the limo fish on? He waves, I'm swimming in the mirror
Won't you roll with the limo fish on? He waves, I'm swimming in the mirror
Sometimes all I think about is you Late nights in the middle of June He waves,
been freaking me out Can't make you happier now Sometimes all I think about is
you Late nights in the middle of June He waves, been freaking me out Can't make
you happier now Usually I put something on TV So we never think about you and me
But today I see our reflections clearly in Hollywood Laying on the screen You
just need a better life in this You need something I can never give Fake water
on across the road It's gone now the night is gone but Sometimes all I think
about is you Late nights in the middle of June He waves, been freaking me out
Can't make you happier now You can't fight it, you can't breathe You say
something so loving but Now I gotta let you go You'll be better off in someone
new I don't wanna be alone You know it hurts

#### Groq API Setup

In [ ]:
!pip install -q groq
import os
from groq import Groq

# Groq Setup
# IMPORTANT: For production, use Colab secrets (left panel '🔑' icon) to store your API key securely.
# For example, store it as 'GROQ_API_KEY' and retrieve it using userdata.get('GROQ_API_KEY')

# Temporarily using the provided key for demonstration. Please replace for production.
os.environ["GROQ_API_KEY"] = "" #add your own api key
client = Groq(api_key=os.environ["GROQ_API_KEY"])

print("Groq client initialized.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 6.0 MB/s eta 0:00:00
Groq client initialized.


#### Retriever Function

In [21]:
# Retriever Function

def retrieve(query, top_k=3):
    # Use the pre-trained SentenceTransformer embedder to encode the query
    # The embeddings need to be of type float32 for FAISS
    q_vec = embedder.encode([query]).astype("float32")
    distances, indices = index.search(q_vec, top_k)
    return [chunks[i] for i in indices[0]]

print("Retriever function defined.")

Retriever function defined.


#### Interactive Q&A with FAISS and Groq

In [17]:
# Install necessary libraries
!pip install -q faiss-cpu scikit-learn groq

import os
import faiss
import numpy as np
from google.colab import files

from sklearn.feature_extraction.text import TfidfVectorizer
from groq import Groq

print("Upload TXT file for FAISS RAG (e.g., my_history.txt)")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

with open(file_name, "r", encoding="utf-8") as f:
    text = f.read()

print("File Loaded successfully.")

Upload TXT file for FAISS RAG (e.g., my_history.txt)


Saving my_histroy.txt.txt to my_histroy.txt.txt
File Loaded successfully.


#### Chunking the Text

In [18]:
# Chunking

chunk_size = 400 # Define chunk size (e.g., number of words)
words = text.split()

chunks = [
    " ".join(words[i:i+chunk_size])
    for i in range(0, len(words), chunk_size)
]

print("Total Chunks:", len(chunks))

Total Chunks: 152


#### Generating TF-IDF Embeddings and Building FAISS Index

In [19]:
# Install necessary dependencies for FAISS and the Sentence Transformers library
!apt install -q libomp-dev # Required for FAISS
!pip install -q sentence-transformers faiss-cpu

import os
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer # Using SentenceTransformer instead of JinaEmbeddings
from google.colab import userdata # To access Colab secrets (though JINA_API_KEY won't be needed for SentenceTransformers)

# NOTE: JINA_API_KEY is no longer needed with SentenceTransformer models that run locally.
# You can remove the JINA_API_KEY setup if you're not using Jina embeddings.
# For now, I'll comment out the JINA_API_KEY specific code.

# jina_api_key = os.environ.get("JINA_API_KEY") or userdata.get("JINA_API_KEY")
# if not jina_api_key:
#     print("JINA_API_KEY not set, proceeding without it (not needed for SentenceTransformers).")
# else:
#     os.environ["JINA_API_KEY"] = jina_api_key
#     print("JINA_API_KEY loaded.")

# Initialize Sentence Transformer Embedding model
# 'all-MiniLM-L6-v2' is a good general-purpose model.
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Generate Embeddings
print(f"Generating embeddings using Sentence Transformer model...")
# The 'chunks' variable is a list of strings created in the previous cell.
embeddings_list = embedder.encode(chunks, show_progress_bar=True)
embeddings = np.array(embeddings_list).astype("float32")

dim = embeddings.shape[1]
print(f"Embeddings generated. Dimension: {dim}")

# FAISS Index
# Re-initialize FAISS index with the new dimension
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

print("FAISS Index Ready with Sentence Transformer Embeddings.")

Reading package lists...
Building dependency tree...
Reading state information...
libomp-dev is already the newest version (1:14.0-55~exp2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Generating embeddings using Sentence Transformer model...


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Embeddings generated. Dimension: 384
FAISS Index Ready with Sentence Transformer Embeddings.


### Alternative: RAG with FAISS and Groq (Step-by-Step)

In [22]:
# Ask Loop

print("\nFAISS DB is Ready. Type your query or 'exit' to quit.\n")

while True:
    query = input("Ask (or exit): ")
    if query.lower() == "exit":
        break

    docs = retrieve(query)
    context = "\n\n".join(docs)

    prompt = f"""
Answer ONLY using the context below:

Context:
{context}

Question: {query}

Answer:
"""

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    print("\nAnswer:\n", response.choices[0].message.content)


FAISS DB is Ready. Type your query or 'exit' to quit.

Ask (or exit): say me something from the document

Answer:
 You equate "Love" with "Utility." You believe you only matter if you are doing something for someone.
Ask (or exit): is ture though ?

Answer:
 The question is: "is ture though ?"

Answer: No, it's not true. The critic is saying that the protagonist's plan to post the screenshots and destroy the girl's reputation is not a good idea because it will lead to chaos and exhaustion. The critic is advising the protagonist to keep the power and use it strategically, rather than using it for revenge.
Ask (or exit): exit
